# 3.2 Assignment

### EDA

In [3]:
# Importing libraries
import pandas as pd

# Importing csv file into pandas dataframe
df = pd.read_csv("dodgers-2022.csv")

df.head()

,month,day,attend,day_of_week,opponent,temp,skies,day_night,cap,shirt,fireworks,bobblehead
0,APR,10,56000,Tuesday,Pirates,67,Clear,Day,NO,NO,NO,NO
1,APR,11,29729,Wednesday,Pirates,58,Cloudy,Night,NO,NO,NO,NO
2,APR,12,28328,Thursday,Pirates,57,Cloudy,Night,NO,NO,NO,NO
3,APR,13,31601,Friday,Padres,54,Cloudy,Night,NO,NO,YES,NO
4,APR,14,46549,Saturday,Padres,57,Cloudy,Night,NO,NO,NO,NO


In [4]:
# Converting boolean variables to binary

# List of boolean variables
boolean_vars = ['cap', 'shirt', 'fireworks', 'bobblehead']

# Replacing YES and NO with 1 and 0
df[boolean_vars] = df[boolean_vars].replace({'YES': 1, 'NO': 0})

# Data preview
df.head()

/var/folders/xh/6p519lr15m14xl0whx7dxdx80000gn/T/ipykernel_5804/3127824243.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[boolean_vars] = df[boolean_vars].replace({'YES': 1, 'NO': 0})


,month,day,attend,day_of_week,opponent,temp,skies,day_night,cap,shirt,fireworks,bobblehead
0,APR,10,56000,Tuesday,Pirates,67,Clear,Day,0,0,0,0
1,APR,11,29729,Wednesday,Pirates,58,Cloudy,Night,0,0,0,0
2,APR,12,28328,Thursday,Pirates,57,Cloudy,Night,0,0,0,0
3,APR,13,31601,Friday,Padres,54,Cloudy,Night,0,0,1,0
4,APR,14,46549,Saturday,Padres,57,Cloudy,Night,0,0,0,0


In [5]:
''' Starting the process of one-hot encoding categorical variables
This is probably overkill, but it was a good exercise.
It also allows us to study the categorical variables inordirinally. I might have made up a word there.'''

# Checking how many unique columns we'll be making.
month_u = df['month'].unique()
day_of_week_u = df['day_of_week'].unique()
opponent_u = df['opponent'].unique()
skies_u = df['skies'].unique()
day_night_u = df['day_night'].unique()

print(month_u)
print(day_of_week_u)
print(opponent_u)
print(skies_u)
print(day_night_u)

['APR' 'MAY' 'JUN' 'JUL' 'AUG' 'SEP' 'OCT']
['Tuesday' 'Wednesday' 'Thursday' 'Friday' 'Saturday' 'Sunday' 'Monday']
['Pirates' 'Padres' 'Braves' 'Nationals' 'Giants' 'Rockies' 'Snakes'
 'Cardinals' 'Astros' 'Brewers' 'Angels' 'White Sox' 'Mets' 'Reds'
 'Phillies' 'Cubs' 'Marlins']
['Clear ' 'Cloudy']
['Day' 'Night']


In [6]:
# Defining categorical variables
cat_vars = ['month', 'day_of_week', 'opponent', 'skies', 'day_night']

# Using one-hot encloding and pivoting out categorical columns as boolean fields
df_encoded = pd.get_dummies(df, columns=cat_vars, drop_first=True)

# Converting to integer for binary instead of t/f
df_encoded = df_encoded.astype(int)

# Data preview
print(df_encoded.head())

   day  attend  temp  cap  shirt  fireworks  bobblehead  month_AUG  month_JUL  \
0   10   56000    67    0      0          0           0          0          0   
1   11   29729    58    0      0          0           0          0          0   
2   12   28328    57    0      0          0           0          0          0   
3   13   31601    54    0      0          1           0          0          0   
4   14   46549    57    0      0          0           0          0          0   

   month_JUN  ...  opponent_Nationals  opponent_Padres  opponent_Phillies  \
0          0  ...                   0                0                  0   
1          0  ...                   0                0                  0   
2          0  ...                   0                0                  0   
3          0  ...                   0                1                  0   
4          0  ...                   0                1                  0   

   opponent_Pirates  opponent_Reds  opponent_Rocki

In [18]:
# Importing library
import statsmodels.api as sm

# Defining target variable
target = 'attend'

# Grouping columns, I got some crazy results without this. How can a game in two non-consecutive months?
groups = {
    'month': [col for col in df_encoded.columns if col.startswith('month_')],
    'day_of_week': [col for col in df_encoded.columns if col.startswith('day_of_week_')],
    'opponent': [col for col in df_encoded.columns if col.startswith('opponent_')],
    'skies': ['skies_Cloudy'],
    'day_night': ['day_night_Night'],
    'bobblehead': ['bobblehead'],
    'shirt': ['shirt'],
    'cap': ['cap'],
    'fireworks': ['fireworks'],
}

# Defining function for stepwise regression
def forward_stepwise_grouped(data, target, groups, criterion='aic'):
    # Initializing the rest of the variables
    remaining_groups = set(groups.keys())
    # List to store best scores
    selected_groups = []
    current_score, best_new_score = float('inf'), float('inf')
    # List to store scores from loop
    scores = []

    # Loop to find best scores until no improvement is found
    while remaining_groups and current_score == best_new_score:
        # List to store scores from each group
        scores_with_candidates = []
        # Evaluation of remaining groups
        for group in remaining_groups:
            group_predictors = groups[group]
            model_predictors = [col for g in selected_groups for col in groups[g]] + group_predictors
            X = sm.add_constant(data[model_predictors])
            y = data[target]
            model = sm.OLS(y, X).fit()
            # Calculating for AIC, BIC and R2
            if criterion == 'aic':
                score = model.aic
            elif criterion == 'bic':
                score = model.bic
            else:
                score = -model.rsquared_adj

            scores_with_candidates.append((score, group))
        
        # Sorting to select best scores
        scores_with_candidates.sort()
        best_new_score, best_group = scores_with_candidates[0]
        
        if best_new_score < current_score:
            remaining_groups.remove(best_group)
            selected_groups.append(best_group)
            current_score = best_new_score
            scores.append((current_score, list(selected_groups)))

    # Final model
    final_predictors = [col for group in selected_groups for col in groups[group]]
    final_model = sm.OLS(data[target], sm.add_constant(data[final_predictors])).fit()
    return final_model, scores

# Stepwise regression
final_model, grouped_scores = forward_stepwise_grouped(df_encoded, target, groups, criterion='aic')

# Display results
print("Final Model Summary:")
print(final_model.summary())

print("\nGrouped Stepwise Selection Scores:")
for score, selected_groups in grouped_scores:
    print(f"AIC: {score}, Groups: {selected_groups}")

Final Model Summary:
                            OLS Regression Results                            
Dep. Variable:                 attend   R-squared:                       0.616
Model:                            OLS   Adj. R-squared:                  0.521
Method:                 Least Squares   F-statistic:                     6.429
Date:                Thu, 12 Dec 2024   Prob (F-statistic):           2.87e-08
Time:                        22:33:02   Log-Likelihood:                -806.54
No. Observations:                  81   AIC:                             1647.
Df Residuals:                      64   BIC:                             1688.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const    

#### This stepwise model shows the variables that have the most impact using a forward stepwise analysis.
#### I tried a stepwise analysis that compared all possible combinations, but it wouldn't stop running. :P

In [20]:
coefficients = final_model.params.round(2).to_dict()
for predictor, coef in coefficients.items():
    print(f"{predictor}: {coef}")

const: 22232.78
bobblehead: 10700.15
shirt: 5964.53
day_of_week_Monday: 10931.43
day_of_week_Saturday: 17883.96
day_of_week_Sunday: 18225.21
day_of_week_Thursday: 12245.21
day_of_week_Tuesday: 19352.01
day_of_week_Wednesday: 12463.74
fireworks: 16309.21
month_AUG: 3341.07
month_JUL: 2564.26
month_JUN: 6933.35
month_MAY: -1924.75
month_OCT: 221.82
month_SEP: -102.94
cap: -5784.56


#### Using what we have learned from the stepwise AIC scores and the coefficients, we can predict which games may have higher attendance.
#### A game (22,233) giving away bobbleheads (10,700) and shirts (5,965) on a Tuesday (19,352) with fireworks (16,309) in June (6,933) would give you the best chance at filling your stadium.
#### I say you could sum those coefficients to give a predicitive value, but then you may get into nonsensical predictions like more attendance than available seats.
#### A surprising result is the coefficient of the cap. I'm guessing they picked really lousy days to give away caps. Like I tell my folks, context is everything. You have to understand the origins of the data.
#### A heat map is a good way to visualize coefficients, but I've got to stop so I can eventually sleep. :) I love this stuff.